8,1

In [9]:
# pip install pyprindが必要
# 大きなファイルを読み込むので４分くらいかかる。
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(50000)
labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for  s in ('test' , 'train'):
    for l in ('pos', 'neg'):
        path = '../../aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()
df .columns = ['review', 'sentiment']

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:53


In [1]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('../../movie_data.csv', index=False)

NameError: name 'df' is not defined

In [3]:
import pandas as pd
df = pd.read_csv('../../movie_data.csv')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


8.2.1

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count =CountVectorizer()
docs = np.array(['The sun is shining','The weather is sweet' , 'The sun is shining, the weather is sweet , and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(count.get_feature_names())
print(bag.toarray())

{'and': 0, 'weather': 8, 'shining': 3, 'is': 1, 'sweet': 5, 'two': 7, 'sun': 4, 'the': 6, 'one': 2}
['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather']
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


8.2.2

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


8.2.3

In [6]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [7]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>' , '',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor("</a>This :) is :( a test :-)!"))

is seven title brazil not available
this is a test :) :( :)


In [8]:
df['review'] = df['review'].apply(preprocessor)

8.2.4

In [9]:
def tokenizer(text):
    return text.split()

print(tokenizer('runners like running and thus they run'))

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']


In [10]:
# pip install nltkが必要
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter('runners like running and thus they run'))

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


In [11]:
#ストップワードの利用
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
[ w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ishii0514/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

8.2.5

In [12]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[250000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values


In [13]:
# ###時間が非常にかかる###
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
param_grid = [
            {'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer':[tokenizer, tokenizer_porter],
              'clf__penalty':['l1','l2'],
              'clf__C':[1.0, 10.0, 100.0]},
            {'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer':[tokenizer, tokenizer_porter],
              'vect__use_idf':[False],
              'vect__norm':[None],
              'clf__penalty':['l1','l2'],
              'clf__C':[1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy', cv=5,verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 33.4min


KeyboardInterrupt: 

In [ ]:
print('Best paramter set: %s', % gs_lr_tfidf.best_params_)

In [ ]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimeter_
print('Test Accuracy: %.3f' % clf.score(X_test,y_test))

8.3

In [17]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>' , '',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[: -3], int(line[-2])
            yield text, label

# 確認
#print(next(stream_docs(path='../../movie_data.csv')))

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
         for _ in range(size):
                text, label = next(doc_stream)
                docs.append(text)
                y.append(label)
    except StopIteration:
        return None, None
    return docs, y


In [18]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None,tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='../../movie_data.csv')

In [28]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()
    

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:59


In [29]:
X_test, y_test = get_minibatch(doc_stream, 5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.867


In [30]:
# テストで使ったデータを使ってもでるを更新
clf = clf.partial_fit(X_test, y_test)